In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Streaming Example').getOrCreate()

ModuleNotFoundError: No module named 'findspark'

In [2]:
from time import sleep

# Read the schema of incoming data
static = spark.read.json('./data/activity-data')
static.printSchema()

AnalysisException: Path does not exist: file:/home/girish/Documents/mahe/Lectures/Deeapk sir- ABD/spark/Class Programs/data/activity-data;

In [3]:
dataSchema = static.schema

# Create streaming object by passing the inferred schema
# Reads one file for each trigger
streaming = spark.readStream.schema(dataSchema).option('maxFilesPerTrigger', 1)\
   .json('./data/activity-data')

# groupBy(): performing transformation
activityCounts = streaming.groupBy('gt').count()
print(type(activityCounts)) 


# Set number of partitio, 5ns for shuffle operations is 5
spark.conf.set('Spark.sql.shuffle.partitions', 5)

activityQuery = activityCounts.writeStream.queryName('activity_counts')\
   .format('memory').outputMode('complete')\
   .start()
print(type(activityQuery))

for x in range(5):
    spark.sql('SELECT * FROM activity_counts').show()
    sleep(10)
activityQuery.awaitTerminator()

NameError: name 'static' is not defined